# Implementation of projective transformations

## Naive algorithm

In this notebook we will implement algorithms for determining projective transformation for given original points (at least 4 points) and their images. 
<br/><br/>
Firstly, we will implement **naive algorithm**. It's based on composition of two transformations: inverted transformation from *basic points* (points with coordinates (1,0,0), (0,1,0), (0,0,1), (1,1,1)) to originals and transformation from basic points to images. They applied together are actually just this transformation.

In [1]:
# imports
import numpy as np

In [2]:
# inits
names = ['A', 'B', 'C', 'D']
coords = ['1. coordinate ', '2. coordinate ', '3. coordinate ']
points_originals = []
points_pictures  = []

In [3]:
# utility method for entering points
# arguments: 
#    - msg -- global message for whole input of points
#    - foreach_msg -- message for each point
#    - list of names of given points
def input_points(msg, foreach_msg, names = names):
    points = []
    print(msg)
    
    for point in names:
        print(foreach_msg, end = " ")
        print("{} ".format(point))
        first_c = float(input(coords[0]))
        second_c = float(input(coords[1]))
        third_c = float(input(coords[2]))
    
        points.append([first_c, second_c, third_c])
    return points

In [4]:
# we enter original points
points_originals = input_points("   Enter input points: ",
                                "* POINT")
points_originals

   Enter input points: 
* POINT A 
1. coordinate -3
2. coordinate -1
3. coordinate 1
* POINT B 
1. coordinate 3
2. coordinate -1
3. coordinate 1
* POINT C 
1. coordinate 1
2. coordinate 1
3. coordinate 1
* POINT D 
1. coordinate -1
2. coordinate 1
3. coordinate 1


[[-3.0, -1.0, 1.0], [3.0, -1.0, 1.0], [1.0, 1.0, 1.0], [-1.0, 1.0, 1.0]]

In [5]:
# we enter images of points
points_pictures = input_points("    Now enter appropriate images of these points",
                                "* IMAGE OF POINT")
points_pictures

    Now enter appropriate images of these points
* IMAGE OF POINT A 
1. coordinate -2
2. coordinate -1
3. coordinate 1
* IMAGE OF POINT B 
1. coordinate 2
2. coordinate -1
3. coordinate 1
* IMAGE OF POINT C 
1. coordinate 2
2. coordinate 1
3. coordinate 1
* IMAGE OF POINT D 
1. coordinate -2
2. coordinate 1
3. coordinate 1


[[-2.0, -1.0, 1.0], [2.0, -1.0, 1.0], [2.0, 1.0, 1.0], [-2.0, 1.0, 1.0]]

In [6]:
from copy import deepcopy

In [7]:
# repeated part of Cramer's rule
# utility method for create matrix delta_i, calcuating determinate, finding lambda - i
# arguments:
#    - matrix of system (coefficients with letters in system)
#    - constants (numbers from other side of equations)
#    - for witch letter we want to know value (witch lambda, expressed by order number)
def lambda_i(matrix_of_system, constants, i):
    delta_i = np.transpose(deepcopy(matrix_of_system))
    delta_i[i] = constants
    delta_i = np.transpose(np.array(delta_i))
    
    return np.linalg.det(delta_i) / np.linalg.det(np.array(matrix_of_system))

In [8]:
# method of whole Cramer's rule, we solve whole system - find values of all 3 lambdas
# matrix dimensions: (3x3), constants (1x3)
def cramers_rule_3(matrix_of_system, constants):
    lambda_1 = lambda_i(matrix_of_system, constants, 0)
    lambda_2 = lambda_i(matrix_of_system, constants, 1)
    lambda_3 = lambda_i(matrix_of_system, constants, 2)
    
    return (lambda_1, lambda_2, lambda_3)
    

In [9]:
# we find projection transformation witch
# maps from base points ((1,0,0), (0,1,0), (0,0,1), (1,1,1))
# to points given as argument (ALWAYS 4 points!)
def projection_transformation_from_base(points):
    matrix_of_system = np.array(points[:-1])
    matrix_of_system = np.transpose(matrix_of_system)
    lambdas = cramers_rule_3(matrix_of_system, points[-1])
    transformation = []
    
    for i in range(0,3):
        transformation.append([lambdas[i] * coordinate for coordinate in points[i]])
    
    return np.transpose(np.array(transformation))

In [10]:
# Finally, we calculate matrix of whole projective transformation
# from points_originals to points_pictures like composition
# of previous tranformation of points_originals (inverted!) and points_pictures
def projection_transformation(points_originals, points_pictures):
    g = projection_transformation_from_base(points_originals)
    f = projection_transformation_from_base(points_pictures)
    
    return np.dot(f, np.linalg.inv(g))

In [11]:
# Matrix of transformation
final_function = projection_transformation(points_originals, points_pictures)
final_function

array([[ 2.,  0.,  0.],
       [ 0.,  2., -1.],
       [ 0., -1.,  2.]])

## DLT - algorithm

**Naive algorithm** for calculating projective transformation for given 4 points and their images isn't bad in terms of time and memory (like its name maybe says), even it's great in this terms. But, it works just for 4 points and uniquely determines transformation. In general case somewhere we will have noise, so it's welcome we have much more given points for greater precision and algorithm which can do with more than 4 points.
<br/><br/>
Algorithm which can do with arbitrary number of points is **DLT - algorithm**. This is sort of algebraic algorithms based on SVD decomposition which minimizes error.

In [12]:
# utility method for finding rows for big matrix
# for each point and its image
# Arguments: point (original point) and point_picture (image of original point)
# NOTE: We return touple with appropriate rows
# (we calculate them by different formulas)
def M(point, point_picture):
    l1 = [0,0,0,
          -point_picture[2] * point[0], 
          -point_picture[2] * point[1],
          -point_picture[2] * point[2],
          point_picture[1] * point[0],
          point_picture[1] * point[1],
          point_picture[1] * point[2]]
    
    l2 = [point_picture[2] * point[0], 
          point_picture[2] * point[1],
          point_picture[2] * point[2],0,0,0,
         -point_picture[0] * point[0], 
          -point_picture[0] * point[1],
          -point_picture[0] * point[2]]
    return (l1,l2)

In [13]:
# utility method for making big matrix (we will decompose her )
# NOTE: We there suppose same number of original points and their images!
def A_matrix(points_originals, points_pictures):
    A = []
    n = len(points_originals)
    for i in range(0,n):
        hs = M(point=points_originals[i], point_picture=points_pictures[i])
        A.append(hs[0])
        A.append(hs[1])
    return A

In [20]:
# Finally, whole DLT algorithm
# NOTE: We use final output from naive algorithm like valid output
# (bad practice in general case)
# for scaling on approximately values like them
# (this 'damn lambda' in projective transformations)
def DLT(points_originals, points_pictures):
    A = A_matrix(points_originals, points_pictures) # big matrix
    decomposition = np.linalg.svd(np.array(A))  # SVD decomposition of big matrix
    
    # parsing transformation from v matrix
    transformation_ = np.array(decomposition[2][-1]).reshape(3,3)
    transformation_ = matrix * ((final_function[0][0] / transformation_[0][0])) # scaling
    return transformation_

In [21]:
DLT(points_originals, points_pictures)

array([[ 2.00000000e+00,  2.31389284e-16, -9.67726185e-16],
       [-2.07703709e-16,  2.00000000e+00, -1.00000000e+00],
       [-2.07703709e-16, -1.00000000e+00,  2.00000000e+00]])

Now we found out why is this algorithm better for general usage. With more correspodences we have greater precision, and of course, better results. But still there are some assues.
<br/><br/>
Not good fact about this algorithm is non-invariance of changes of coordinates (beacuse this is just an algebraic and no geometry algorithm). Solution is doing *normalisation* of coordinates and then transformation won't depend of choice of coordinates. *Normalisation* is composed of determining center of points, its translating to $origin$ and scaling to average distance of origin $\sqrt{2}$ .